In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
af = pd.read_csv('audio_features.csv')
af.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album
0,0,0.735,0.444,10,-10.519,1,0.0684,0.2040,0.001200,0.1700,0.0984,97.038,audio_features,4g2c7NoTWAOSYDy44l9nub,spotify:track:4g2c7NoTWAOSYDy44l9nub,https://api.spotify.com/v1/tracks/4g2c7NoTWAOS...,https://api.spotify.com/v1/audio-analysis/4g2c...,202396,4,midnights
1,1,0.658,0.378,7,-8.300,1,0.0379,0.0593,0.000000,0.0976,0.0382,108.034,audio_features,199E1RRrVmVTQqBXih5qRC,spotify:track:199E1RRrVmVTQqBXih5qRC,https://api.spotify.com/v1/tracks/199E1RRrVmVT...,https://api.spotify.com/v1/audio-analysis/199E...,218271,4,midnights
2,2,0.638,0.634,4,-6.582,1,0.0457,0.1330,0.000001,0.1520,0.5190,96.953,audio_features,02Zkkf2zMkwRGQjZ7T4p8f,spotify:track:02Zkkf2zMkwRGQjZ7T4p8f,https://api.spotify.com/v1/tracks/02Zkkf2zMkwR...,https://api.spotify.com/v1/audio-analysis/02Zk...,200690,4,midnights
3,3,0.659,0.323,9,-13.425,1,0.0436,0.7350,0.003210,0.1160,0.1540,110.007,audio_features,6ADDIJxxqzM9LMpm78yzQG,spotify:track:6ADDIJxxqzM9LMpm78yzQG,https://api.spotify.com/v1/tracks/6ADDIJxxqzM9...,https://api.spotify.com/v1/audio-analysis/6ADD...,256124,4,midnights
4,4,0.694,0.380,2,-10.307,1,0.0614,0.4160,0.000008,0.1260,0.3760,120.044,audio_features,7gVWKBcfIW93YxNBi3ApIE,spotify:track:7gVWKBcfIW93YxNBi3ApIE,https://api.spotify.com/v1/tracks/7gVWKBcfIW93...,https://api.spotify.com/v1/audio-analysis/7gVW...,194207,4,midnights


In [8]:
mid_val = af[af['album'] == 'midnights']['valence']
lon_val = af[af['album'] == 'loneliest']['valence']

In [10]:
result1 = stats.ttest_ind(mid_val, lon_val)

In [11]:
result1.statistic

-4.205937943601481

In [17]:
round(result1.pvalue, 5)

0.00018

In [18]:
mid_dance = af[af['album'] == 'midnights']['danceability']
lon_dance = af[af['album'] == 'loneliest']['danceability']

In [19]:
result2 = stats.ttest_ind(mid_dance, lon_dance)

In [20]:
result2.statistic

-2.1273893857382227

In [21]:
result2.pvalue

0.04072264959445446

In [27]:
modexalbum_freq = af.groupby(['album', 'mode']).count()['id']

In [35]:
result3 = stats.chisquare(modexalbum_freq)

In [30]:
result3.statistic

18.22222222222222

In [32]:
result3.pvalue

0.0003957850029937146

In [37]:
af.groupby(['album', 'key']).count()['id']

album      key
loneliest  0      2
           1      1
           2      2
           4      1
           5      1
           6      3
           7      2
           8      2
           9      2
midnights  0      4
           2      1
           4      3
           5      1
           6      1
           7      5
           8      1
           9      1
           10     2
           11     1
Name: id, dtype: int64

In [38]:
stats.chisquare(af.groupby(['album', 'key']).count()['id'])

Power_divergenceResult(statistic=12.555555555555557, pvalue=0.8172706096483118)